# Задание 1

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

SyntaxError: invalid syntax (1992139135.py, line 11)

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [ ]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [1]:
def __sub__(self, other: Union['Dual', float]) -> 'Dual':
    match other:
        case Dual(value, d):
            return Dual(self.value - o_value, self.d - o_d)
        case Number():
            return Dual(self.value - float(other), self.d)
def __div__(self, other: Union['Dual', float]) -> 'Dual':
    match other:
        case Dual(value, d):
            return Dual(self.value / value, (self.d * o_value - o_d * self.value) / (o_value ** 2))
        case Number():
            return Dual(self.value / float(other), self.d / float(other))
def diff(func: Callable[[Dual], Dual]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.)).d

SyntaxError: invalid syntax (2147652815.py, line 4)

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [2]:
import math

def exp(x: Union[Dual, float]) -> Dual:
    match x:
        case Dual(value, d):
            return Dual(math.exp(value), math.exp(value) * d)
        case float(number):
            return Dual(math.exp(number), 0.)

def cos(x: Union[Dual, float]) -> Dual:
    match x:
        case Dual(value, d):
            return Dual(math.cos(value), -math.sin(value) * d)
        case float(number):
            return Dual(math.cos(number), 0.)

def sin(x: Union[Dual, float]) -> Dual:
    match x:
        case Dual(value, d):
            return Dual(math.sin(value), math.cos(value) * d)
        case float(number):
            return Dual(math.sin(number), 0.)

def log(x: Union[Dual, float], base: float = math.e) -> Dual:
    match x:
        case Dual(value, d):
            return Dual(math.log(value, base), d / (value * math.log(base)))
        case float(number):
            return Dual(math.log(number, base), 0.)


def derivative(func: Callable[[Dual], Dual], x: float, delta: float = 1e-8) -> float:
    return (func(Dual(x)) - func(Dual(x - delta))).value / delta

SyntaxError: invalid syntax (1836123469.py, line 4)

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [ ]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

In [2]:
import sympy as sy

x = sy.symbols('x')
d = 5 * x * x + 2 * x + 2
func_x = sy.diff(d,x)
func_x_value = float(func_x.evalf(subs={x:2}))
print(func_x_value)

22.0


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [3]:
import sympy as sy

def function_f(x,y,z):
    return x * y + z - 5 * y

x,y,z = sy.symbols('x y z')

def function_d(a,b,c):
    diff_x = sy.diff(function_f(x, y, z), x)
    diff_y = sy.diff(function_f(x, y, z), y)
    diff_z = sy.diff(function_f(x, y, z), z)
    diff_x_r = int(diff_x.evalf(subs={x: a, y: b, z: c}))
    diff_y_r = int(diff_y.evalf(subs={x: a, y: b, z: c}))
    diff_z_r = int(diff_z.evalf(subs={x: a, y: b, z: c}))
    return diff_x_r, diff_y_r, diff_z_r

res = function_d(10,10,10)
print(res)


(10, 5, 1)
